MODEL SIZE IS **2.24 GB** ONLY, BUT EXCELS GREATLY AT ALL NLP TASKS!

Choose **Runtime  ->  Change runtime type  ->  T4 GPU (HIT SAVE)**

Hit **Ctrl + F9** to run all cells in 1-click.

In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer
import os
from threading import Thread

try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Check if running in Colab and set save directory accordingly
if IN_COLAB:
    drive.mount('/content/drive')
    save_directory = '/content/drive/MyDrive/llama_model'
else:
    save_directory = 'llama_model'  # Or any local path you prefer

if not os.path.exists(save_directory):
    print("Model not found in Drive. Downloading from Hugging Face...")
    tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-3B-Instruct-bnb-4bit")
    model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-3B-Instruct-bnb-4bit")

    os.makedirs(save_directory, exist_ok=True)

    tokenizer.save_pretrained(save_directory)
    model.save_pretrained(save_directory)
else:
    print("Model found in Drive. Loading...")
    tokenizer = AutoTokenizer.from_pretrained(save_directory)
    model = AutoModelForCausalLM.from_pretrained(save_directory)

Mounted at /content/drive
Model found in Drive. Loading...


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from IPython.display import HTML, display

def prompt(prompt):

    display(HTML("""
    <style>
    pre {
        white-space: pre-wrap;       /* Wrap long lines */
        word-wrap: break-word;       /* Break words if necessary */
        width: 70% !important; /* Adjust the percentage as needed */
    }
    </style>
    """))

    model.generation_config.pad_token_id = tokenizer.pad_token_id
    prompt = {prompt}

    messages = [
        {"role": "system", "content": "Give concise answers!"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)


    # Create a streamer object
    streamer = TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    # Pass the streamer to the generate method
    generation_kwargs = dict(
        **model_inputs,
        streamer=streamer,
        max_new_tokens=5000,
    )
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # Iterate over the generated tokens and print them
    for new_text in streamer:
        print(new_text, end="", flush=True)

    thread.join()

In [5]:
def prompt_value():
  input_value = input("Enter your prompt: ")
  print("\nResponse: ")
  prompt(input_value)

In [6]:
prompt_value()

Enter your prompt: what is deep learning

Response: 


Deep learning is a subset of machine learning that uses artificial neural networks to analyze and interpret data. It's a type of AI that's particularly good at recognizing patterns in large datasets, such as images, speech, and text.